In [1]:
from pyspark import SparkFiles

In [2]:
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df =  spark.read.option('header', 'true').csv("file://"+SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), inferSchema=True, sep='\t')
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
marketplace|customer_id| review_id|product_id|product_parent| product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase| review_headline| review_body| review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
 US| 3653882|R3O9SGZBVQBV76|B00FALQ1ZC| 937001370|Invicta Women's 1...| Watches| 5| 0| 0| N| Y| Five Stars|Absolutely love t...|2015-08-31 00:00:00|
 US| 14661224| RKH8BNC3L5DLF|B00D3RGO20| 484010722|Kenneth Cole New ...| Watches| 5| 0| 0| N| Y|I love thiswatch ...|I love this watch...|2015-08-31 00:00:00|
 US| 27324930|R2HLE8WKZSU3NL|B00DKYC7TK| 361166390|Ritche 22mm Black...| Watches| 2| 1| 1| N| Y| Two Stars| Scratches|2015-08-31 00:00:00|
 US| 7211452|R31U3UH5AZ42LL|B000EQS1JW| 958035625|Citizen Men's BM8...| Watches| 5| 0| 0| N| Y| Five Stars|It works well on ...|2015-08-31 00:00:00|
 US| 12733322|R2SV659OUJ945Y|B00A6GFD7S| 765328221|Orient ER27009B M...| Watches| 4| 0| 0| N| Y|Beautiful face, b...|Beautiful watch f...|2015-08-31 00:00:00|
 US| 6576411| RA51CP8TR5A2L|B00EYSOSE8| 230493695|Casio Men's GW-94...| Watches| 5| 0| 0| N| Y| No complaints|i love this watch...|2015-08-31 00:00:00|
 US| 11811565| RB2Q7DLDN6TH6|B00WM0QA3M| 549298279|Fossil Women's ES...| Watches| 5| 1| 1| N| Y| Five Stars|for my wife and s...|2015-08-31 00:00:00|
 US| 49401598|R2RHFJV0UYBK3Y|B00A4EYBR0| 844009113|INFANTRY Mens Nig...| Watches| 1| 1| 5| N| N|I was about to bu...|I was about to bu...|2015-08-31 00:00:00|
 US| 45925069|R2Z6JOQ94LFHEP|B00MAMPGGE| 263720892|G-Shock Men's Gre...| Watches| 5| 1| 2| N| Y| Perfect watch!|Watch is perfect....|2015-08-31 00:00:00|
 US| 44751341| RX27XIIWY5JPB|B004LBPB7Q| 124278407|Heiden Quad Watch...| Watches| 4| 0| 0| N| Y|Great quality and...|Great quality and...|2015-08-31 00:00:00|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
only showing top 10 rows

In [3]:
# Count 
df.count()

Out[3]: 960872

In [4]:
#
df_select = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
df_select.show()

+--------------+-----------+-------------+-----------+----+
 review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
R3O9SGZBVQBV76| 5| 0| 0| N|
 RKH8BNC3L5DLF| 5| 0| 0| N|
R2HLE8WKZSU3NL| 2| 1| 1| N|
R31U3UH5AZ42LL| 5| 0| 0| N|
R2SV659OUJ945Y| 4| 0| 0| N|
 RA51CP8TR5A2L| 5| 0| 0| N|
 RB2Q7DLDN6TH6| 5| 1| 1| N|
R2RHFJV0UYBK3Y| 1| 1| 5| N|
R2Z6JOQ94LFHEP| 5| 1| 2| N|
 RX27XIIWY5JPB| 4| 0| 0| N|
R15C7QEZT0LGZN| 4| 2| 2| N|
R361XSS37V0NCZ| 1| 0| 0| N|
 ROTNLALUAJAUB| 3| 0| 0| N|
R2DYX7QU6BGOHR| 5| 0| 0| N|
 RWASY7FKI7QOT| 5| 2| 3| N|
R2KKYZIN3CCL21| 5| 0| 0| N|
R22H4FGVD5O52O| 4| 1| 1| N|
R11UACZERCM4ZY| 5| 0| 0| N|
R1AT8NQ38UQOL6| 5| 0| 0| N|
R2NCZRQGIF1Q75| 4| 0| 0| N|
+--------------+-----------+-------------+-----------+----+
only showing top 20 rows

In [5]:
df_vote = df_select.filter(df_select['total_votes'] >= 10)
df_vote_select = df_vote.filter(df_vote["helpful_votes"]/df_vote["total_votes"] >= 0.5)

In [6]:
vine_df = df_vote_select.filter(df_vote_select['vine']== 'Y')
nvine_df = df_vote_select.filter(df_vote_select['vine']== 'N')

vine_df.describe().show()
nvine_df.describe().show()

+-------+--------------+-----------------+------------------+-----------------+----+
summary| review_id| star_rating| helpful_votes| total_votes|vine|
+-------+--------------+-----------------+------------------+-----------------+----+
 count| 121| 121| 121| 121| 121|
 mean| null|4.008264462809917|23.520661157024794|26.71900826446281|null|
 stddev| null|0.987386008345563| 35.61228701201166| 38.1379564083901|null|
 min|R131OFSBT1U9QT| 1| 6| 10| Y|
 max| RXT80TR6U2APE| 5| 349| 370| Y|
+-------+--------------+-----------------+------------------+-----------------+----+

+-------+--------------+------------------+-----------------+-----------------+-----+
summary| review_id| star_rating| helpful_votes| total_votes| vine|
+-------+--------------+------------------+-----------------+-----------------+-----+
 count| 23615| 23615| 23615| 23615|23615|
 mean| null|3.7885665890323947|22.44984120262545|25.18733855600254| null|
 stddev| null|1.5217476148482418|46.49478539466279|50.41106170740236| null|
 min|R1000IUWCC6GL8| 1| 5| 10| N|
 max| RZZTS0FEE75XY| 5| 4004| 4249| N|
+-------+--------------+------------------+-----------------+-----------------+-----+

In [7]:
from pyspark.sql.functions import col, avg
# number of five star reviews
vine_five_star_number = vine_df.select("star_rating").filter(vine_df['star_rating'] == 5).count()
# number of reviews
vine_number = vine_df.count()

print(vine_number)
print(vine_five_star_number)
# average rating
vine_average_rating = vine_df.agg(avg(col("star_rating"))).show()
# number of helpful votes
vine_helpful_votes = vine_df.select("helpful_votes").groupBy().sum().show()
# Percent five star rating
percentage_five_star_vine = vine_five_star_number / vine_number
print(percentage_five_star_vine)

121
43
+-----------------+
 avg(star_rating)|
+-----------------+
4.008264462809917|
+-----------------+

+------------------+
sum(helpful_votes)|
+------------------+
 2846|
+------------------+

0.35537190082644626

In [8]:
# number of five star reviews
nvine_five_star_number = nvine_df.select("star_rating").filter(nvine_df['star_rating'] == 5).count()
# number of reviews
nvine_number = nvine_df.count()

print(nvine_number)
print(nvine_five_star_number)
# average rating
nvine_average_rating = nvine_df.agg(avg(col("star_rating"))).show()
# number of helpful votes
nvine_helpful_votes = nvine_df.select("helpful_votes").groupBy().sum().show()
# Percent five star rating
percentage_five_star_nvine = nvine_five_star_number / nvine_number
print(percentage_five_star_nvine)

23615
12083
+------------------+
 avg(star_rating)|
+------------------+
3.7885665890323947|
+------------------+

+------------------+
sum(helpful_votes)|
+------------------+
 530153|
+------------------+

0.5116663137836122